In [ ]:
import os
import torch
from torch import nn as nn
from torchvision import datasets, models, transforms

In [ ]:
ddir = '/content/drive/MyDrive/study/study by lecture/implementation/dataset/hym_data'

batch_size = 64
num_workers = 2

data_transformers = {
    'train' : transforms.Compose(
        [
          transforms.RandomResizedCrop(224),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])

        ]
    ),
    'val' : transforms.Compose(
        [
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    )
}

img_data = {
    k : datasets.ImageFolder(os.path.join(ddir, k), data_transformers[k]) for k in ['train', 'val']
    }

dLoaders = {
    k : torch.utils.data.DataLoader(img_data[k], batch_size = batch_size, shuffle = True, num_workers = num_workers) for k in ['train','val']
}

dset_sizes = {x:len(img_data[x]) for x in ['train','val']}
classes = img_data['train'].classes

dvc = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm2d
import time
import torch.nn as nn

class BasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride = 1):
    super().__init__()
    
    self.residual_function = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
        nn.BatchNorm2d(out_channels),
        nn.ReLu(inplace = True),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1, bias = False),
        nn.BatchNorm2d(out_channels)
    )

    self.shorcut = nn.Sequential()

    if stride != 1 or in_channels != out_channels:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_channels, out_channels, kernel_size=1, stride = stride, bias = False),
          nn.BatchNorm2d(out_channels)
      )

  def forward(self, x):
    return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))


class ResNet(nn.Module):
  def __init__(self, block, num_block, num_classes = 100):
    super().__init__()

    self.conf1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size = 3, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace = True))
    
    self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
    self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
    self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
    self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
    self.avg_pool = nn.AdaptiveAvgPool2d((1,1))

    self.fc = nn.Linear(512, num_classes)

  def _make_layer(self, block, out_channels, num_blocks, stride):

    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_channels, out_channels, stride))
      self.in_channels = out_channels
 

[2, 1]